In [4]:
from IPython.display import HTML
import numpy as np
#import urllib2
import bs4 #this is beautiful soup
import time
import operator
import socket
#import cPickle
import re # regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

#from secret import *

In [5]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']

users = pd.read_csv(
    'http://files.grouplens.org/datasets/movielens/ml-100k/u.user',
    sep='|', names=u_cols)

users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [11]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(
    'http://files.grouplens.org/datasets/movielens/ml-100k/u.data',
    sep='\t', names=r_cols,encoding="utf-8")

ratings.head()
          

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [14]:
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']

movies = pd.read_csv(
    'http://files.grouplens.org/datasets/movielens/ml-100k/u.item',
    sep='|', names=m_cols, usecols=range(5), encoding='ISO-8859–1')

movies.head()

,movie_id,title,release_date,video_release_date,imdb_url
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995)


In [15]:
print(movies.describe())

          movie_id  video_release_date
count  1682.000000                 0.0
mean    841.500000                 NaN
std     485.695893                 NaN
min       1.000000                 NaN
25%     421.250000                 NaN
50%     841.500000                 NaN
75%    1261.750000                 NaN
max    1682.000000                 NaN


In [16]:
users['occupation'].head()

0    technician
1         other
2        writer
3    technician
4         other
Name: occupation, dtype: object

In [17]:
columns_you_want = ['occupation', 'sex']
print(users[columns_you_want].head())

   occupation sex
0  technician   M
1       other   F
2      writer   M
3  technician   M
4       other   F


In [19]:
oldUsers = users[users.age > 25]
oldUsers.head()

,user_id,age,sex,occupation,zip_code
1,2,53,F,other,94043
4,5,33,F,other,15213
5,6,42,M,executive,98101
6,7,57,M,administrator,91344
7,8,36,M,administrator,05201


In [27]:
oldmale = users[(users.age > 40) & (users.sex == "M")]
oldmale.head()
            

,user_id,age,sex,occupation,zip_code
5,6,42,M,executive,98101
6,7,57,M,administrator,91344
9,10,53,M,lawyer,90703
12,13,47,M,educator,29206
13,14,45,M,scientist,55106


In [32]:
users[(users.sex=="F") & (users.occupation=="programmer")].age.mean()

32.166666666666664

In [33]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [41]:
grouped_data = ratings.groupby('user_id')
ratings_per_user = grouped_data.count()
ratings_per_user.head(5)

,movie_id,rating,unix_timestamp
user_id,,,
1,272,272,272
2,62,62,62
3,54,54,54
4,24,24,24
5,175,175,175


In [37]:
grouped_data = ratings['movie_id'].groupby(ratings['user_id'])
ratings_per_user = grouped_data.count()
ratings_per_user.head(5)

user_id
1    272
2     62
3     54
4     24
5    175
Name: movie_id, dtype: int64

In [50]:
grouped_data = ratings['rating'].groupby(ratings['movie_id'])
avg_rating = grouped_data.mean()
avg_rating.head(5)

movie_id
1    3.878319
2    3.206107
3    3.033333
4    3.550239
5    3.302326
Name: rating, dtype: float64

In [86]:
grouped = ratings.groupby(ratings['movie_id'])
average_ratings = grouped['rating'].mean().sort_values(ascending=False)
average_ratings.head(5)


movie_id
1293    5.0
1467    5.0
1653    5.0
814     5.0
1122    5.0
Name: rating, dtype: float64

In [93]:
maximum_rating = average_ratings.max()
ids = average_ratings[average_ratings == maximum_rating].index

In [94]:
movies[movies.movie_id.isin(ids)]

,movie_id,title,release_date,video_release_date,imdb_url
813,814,"Great Day in Harlem, A (1994)",01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Great%20Day%2...
1121,1122,They Made Me a Criminal (1939),01-Jan-1939,NaN,http://us.imdb.com/M/title-exact?They%20Made%2...
1188,1189,Prefontaine (1997),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Prefontaine%2...
1200,1201,Marlene Dietrich: Shadow and Light (1996),02-Apr-1996,NaN,http://us.imdb.com/M/title-exact?Marlene%20Die...
1292,1293,Star Kid (1997),16-Jan-1998,NaN,http://us.imdb.com/M/title-exact?imdb-title-12...
1466,1467,"Saint of Fort Washington, The (1993)",01-Jan-1993,NaN,http://us.imdb.com/M/title-exact?Saint%20of%20...
1499,1500,Santa with Muscles (1996),08-Nov-1996,NaN,http://us.imdb.com/M/title-exact?Santa%20with%...
1535,1536,Aiqing wansui (1994),22-Jul-1996,NaN,http://us.imdb.com/M/title-exact?Aiqing%20Wans...
1598,1599,Someone Else's America (1995),10-May-1996,NaN,http://us.imdb.com/M/title-exact?Someone%20Els...
1652,1653,Entertaining Angels: The Dorothy Day Story (1996),27-Sep-1996,NaN,http://us.imdb.com/M/title-exact?Entertaining%...


In [95]:
how_many_ratings = grouped.count()
print(how_many_ratings[average_ratings == maximum_rating])

          user_id  rating  unix_timestamp
movie_id                                 
814             1       1               1
1122            1       1               1
1189            3       3               3
1201            1       1               1
1293            3       3               3
1467            2       2               2
1500            2       2               2
1536            1       1               1
1599            1       1               1
1653            1       1               1


C:\Users\hwny-yangkp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
